In [1]:
import pandas as pd
df=pd.read_excel("Data/PPI_data/NIHMS754821-supplement-11.xlsx", sheet_name=1,dtype={"Gene_Symbol": str})
df=df.dropna() #we only want measured interactions


c:\Users\joaof\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [2]:
df

,Entrez_Gene_ID,Gene_Symbol,Isoform_ID,Category,Interactor_Entrez_Gene_ID,Interactor_Symbol,Interactor_ID,Interaction_Found
0,81,ACTN4,ACTN4_1,reference,373,TRIM23,TRIM23,negative
1,81,ACTN4,ACTN4_1,reference,51778,MYOZ2,MYOZ2,positive
2,81,ACTN4,ACTN4_4,alternative,373,TRIM23,TRIM23,positive
3,81,ACTN4,ACTN4_4,alternative,51778,MYOZ2,MYOZ2,negative
4,207,AKT1,AKT1_1,reference,8115,TCL1A,TCL1A,positive
...,...,...,...,...,...,...,...,...
2020,387032,ZKSCAN4,ZKSCAN4_3,alternative,84307,ZNF397,ZNF397,positive
2022,387032,ZKSCAN4,ZKSCAN4_3,alternative,84838,ZNF496,ZNF496,positive
2023,387032,ZKSCAN4,ZKSCAN4_3,alternative,158399,ZNF483,ZNF483,positive
2024,387032,ZKSCAN4,ZKSCAN4_3,alternative,342945,ZSCAN22,ZSCAN22,positive


In [3]:
dictio = df["Gene_Symbol"].astype(str).value_counts()

dictio= pd.merge(dictio,df.groupby("Gene_Symbol")["Isoform_ID"].nunique(),on = "Gene_Symbol", how="inner")
dictio=pd.merge(dictio,df.groupby("Gene_Symbol")["Interactor_ID"].nunique(),on = "Gene_Symbol", how="inner")

In [4]:
n=2

filtered = dictio[dictio["count"]>n]

import requests

def gene_to_uniprot(gene_name, organism="9606"):  
    url = "https://rest.uniprot.org/uniprotkb/search"
    params = {
        "query": f"gene_exact:{gene_name} AND organism_id:{organism}",
        "format": "json",
        "fields": "accession,gene_names"
    }
    r = requests.get(url, params=params)
    if r.status_code != 200 or "results" not in r.json():
        return None
    results = r.json()["results"]
    if not results:
        return None
    return results[0]["primaryAccession"]



def get_afdb_quality(uniprot_id):
    url = f"https://alphafold.ebi.ac.uk/api/prediction/{uniprot_id}"
    
    r = requests.get(url)
    if r.status_code != 200:
        return None
    
    data = r.json()[0]  # AFDB returns a list
    
    return {
        "uniprot_id": data["uniprotId"],
        "description": data["uniprotDescription"],
        "fractionPlddtVeryLow" : data["fractionPlddtVeryLow"],
        'fractionPlddtLow' : data["fractionPlddtLow"],
        'fractionPlddtConfident': data['fractionPlddtConfident'],
        'fractionPlddtVeryHigh':data['fractionPlddtVeryHigh'],
        "confidence_avg": data["globalMetricValue"]
    }

print(get_afdb_quality(uniprot_id=gene_to_uniprot("AKT1")))  # AKT1


{'uniprot_id': 'AKT1_HUMAN', 'description': 'RAC-alpha serine/threonine-protein kinase', 'fractionPlddtVeryLow': 0.1, 'fractionPlddtLow': 0.067, 'fractionPlddtConfident': 0.296, 'fractionPlddtVeryHigh': 0.537, 'confidence_avg': 83.06}


In [5]:
filtered

,count,Isoform_ID,Interactor_ID
Gene_Symbol,,,
ZNF688,209,3,75
BANP,142,6,27
HNRPLL,100,3,35
NDN,93,5,23
HGS,83,2,51
...,...,...,...
SERPING1,3,3,1
PRCC,3,3,1
SEC22C,3,3,1


In [6]:
confidence_avg=[]
filtered = filtered.reset_index()  # moves index into a column
for i in range(len(filtered)):
    
    d=get_afdb_quality(uniprot_id=gene_to_uniprot(filtered["Gene_Symbol"][i]))
    d["counts"] = filtered["count"][i]
    d["unique_isoforms"] = filtered["Isoform_ID"][i]
    d["Unique_interactors_tested_on_Gene"] = filtered["Interactor_ID"][i]
    confidence_avg.append(d)



In [7]:
import numpy as np
df_final=pd.DataFrame(confidence_avg)



In [8]:
df_final


,uniprot_id,description,fractionPlddtVeryLow,fractionPlddtLow,fractionPlddtConfident,fractionPlddtVeryHigh,confidence_avg,counts,unique_isoforms,Unique_interactors_tested_on_Gene
0,ZN688_HUMAN,Zinc finger protein 688,0.449,0.188,0.355,0.007,59.03,209,3,75
1,BANP_HUMAN,Protein BANP,0.569,0.095,0.109,0.227,57.88,142,6,27
2,HNRLL_HUMAN,Heterogeneous nuclear ribonucleoprotein L-like,0.285,0.053,0.238,0.423,72.75,100,3,35
3,NECD_HUMAN,Necdin,0.293,0.184,0.495,0.028,66.88,93,5,23
4,HGS_HUMAN,Hepatocyte growth factor-regulated tyrosine ki...,0.401,0.075,0.171,0.352,66.69,83,2,51
...,...,...,...,...,...,...,...,...,...,...
103,IC1_HUMAN,Plasma protease C1 inhibitor,0.258,0.010,0.074,0.658,79.56,3,3,1
104,PRCC_HUMAN,Proline-rich protein PRCC,0.322,0.448,0.177,0.053,59.16,3,3,1
105,SC22C_HUMAN,Vesicle-trafficking protein SEC22c,0.106,0.073,0.294,0.528,82.56,3,3,1
106,SRRM1_HUMAN,Serine/arginine repetitive matrix protein 1,0.664,0.152,0.150,0.034,51.62,3,2,2


In [9]:
df_final[df_final["unique_isoforms"]*df_final["Unique_interactors_tested_on_Gene"]==df_final["counts"]]

,uniprot_id,description,fractionPlddtVeryLow,fractionPlddtLow,fractionPlddtConfident,fractionPlddtVeryHigh,confidence_avg,counts,unique_isoforms,Unique_interactors_tested_on_Gene
10,PIMRE_HUMAN,Protein PIMREG,0.277,0.542,0.126,0.055,59.28,32,2,16
19,DAAF4_HUMAN,Dynein axonemal assembly factor 4,0.071,0.067,0.326,0.536,83.88,18,3,6
20,RBM10_HUMAN,RNA-binding protein 10,0.430,0.106,0.362,0.102,62.16,18,1,18
21,S100B_HUMAN,Protein S100-B,0.011,0.022,0.239,0.728,91.44,16,2,8
22,UPP2_HUMAN,Uridine phosphorylase 2,0.201,0.024,0.040,0.735,83.81,16,4,4
...,...,...,...,...,...,...,...,...,...,...
102,MIPO1_HUMAN,Mirror-image polydactyly gene 1 protein,0.027,0.084,0.218,0.670,88.38,3,3,1
103,IC1_HUMAN,Plasma protease C1 inhibitor,0.258,0.010,0.074,0.658,79.56,3,3,1
104,PRCC_HUMAN,Proline-rich protein PRCC,0.322,0.448,0.177,0.053,59.16,3,3,1
105,SC22C_HUMAN,Vesicle-trafficking protein SEC22c,0.106,0.073,0.294,0.528,82.56,3,3,1
